In [1]:
import pandas as pd

In [2]:
import re
import tqdm

In [3]:
import datetime

In [4]:
zerodha_data_file = "Data/NewsData/var/lib/mysql-files/pulse.csv"

In [5]:
news_line_regex = r'(?P<headline>\"([^\"]|\n)*\"),(?P<url>\"([^\"]|\n)*\"),(?P<timestamp>\"([^\"]|\n)*\")'

In [6]:
pattern = re.compile(news_line_regex)

In [7]:
lines = None
with open(zerodha_data_file) as file:
    lines = file.readlines()

In [8]:
all_lines = []
fail=0
for line in tqdm.tqdm(lines):
    match = pattern.match(line)
    if match:
        news_obj = {}
        news_obj["headline"] = match.group('headline')[1:-1]
        news_obj["url"] = match.group('url')[1:-1]
        timestamp = match.group('timestamp')[1:-1]
        news_obj["timestamp"] = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S").date()
        all_lines.append(news_obj)
    else:
        fail+=1

100%|██████████| 422289/422289 [00:10<00:00, 42044.81it/s]


In [9]:
df = pd.DataFrame(all_lines)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420739 entries, 0 to 420738
Data columns (total 3 columns):
headline     420739 non-null object
timestamp    420739 non-null object
url          420739 non-null object
dtypes: object(3)
memory usage: 9.6+ MB


In [11]:
df_without1970 = df[df["timestamp"] > datetime.date(1970,1,1)]

In [12]:
df_without1970.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330627 entries, 0 to 330626
Data columns (total 3 columns):
headline     330627 non-null object
timestamp    330627 non-null object
url          330627 non-null object
dtypes: object(3)
memory usage: 10.1+ MB


In [13]:
df_without1970["headline_LC"] = df_without1970["headline"].str.lower()

/home/chetan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
hdfc_news = df_without1970[df_without1970["headline_LC"].str.contains("hdfc")]
hdfc_news["ticker"] = ["HDFCBANK"]*len(hdfc_news)
hdfc_news = hdfc_news.drop(["url", "headline_LC"], axis=1)

/home/chetan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
icici_news = df_without1970[df_without1970["headline_LC"].str.contains("icici")]
icici_news["ticker"] = ["ICICIBANK"]*len(icici_news)
icici_news = icici_news.drop(["url", "headline_LC"], axis=1)

/home/chetan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
axis_news = df_without1970[df_without1970["headline_LC"].str.contains("axis")]
axis_news["ticker"] = ["AXISBANK"]*len(axis_news)
axis_news = axis_news.drop(["url", "headline_LC"], axis=1)

/home/chetan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
total_news_df = pd.concat([hdfc_news, icici_news, axis_news], ignore_index=True)

In [18]:
total_news_df.to_csv("FINAL_DATA/bank_news_zerodha_pulse_~.csv", header=False, index=False, sep='~')